In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20000)

import csv

import json
import urllib

# Load cleaned and geo data

In [3]:
df = pd.read_csv('../data/preprocessed/cleaned_data.csv', delimiter=",")
geo_df = pd.read_csv('../data/raw/NEW_Temporary_Container_Locations.csv', delimiter=";", encoding='cp1252')

We can see, that the location is provided as a strees adress

In [4]:
geo_df.head()

,Sensor,Sens. Typ,Location,Cont. Typ,Farbe
0,70B3D500700016DA,Senseno Ultrasonic,An den Roethen 53,Glas (3 Kammern),white
1,70B3D500700016DE,Senseno Ultrasonic,Hungener Str. 6,Glas (3 Kammern),brown
2,70B3D500700016DF,Senseno Ultrasonic,Standort unbekannt,Glas (3 Kammern),white
3,70B3D500700016E0,Senseno Ultrasonic,Gronauer Str. 39,Glas (3 Kammern),green
4,70B3D500700016E5,Senseno Ultrasonic,Wetteraustr. 1,Glas (3 Kammern),green


In [5]:
geo_df['Address'] = geo_df['Location']+', Frankfurt'

# Translate street name into lat lon
Hier nen API Key ziehen: https://console.cloud.google.com/apis/dashboard

In [6]:
from geopy.geocoders import GoogleV3
AUTH_KEY = "API_KEY"
geolocator = GoogleV3(api_key=AUTH_KEY)

In [7]:
geo_df["loc"] = geo_df["Address"].apply(geolocator.geocode)

In [9]:
geo_df["point"]= geo_df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)

In [10]:
geo_df[['lat', 'lon', 'altitude']] = pd.DataFrame(geo_df['point'].to_list(), index=geo_df.index)

In [11]:
geo_df

,Sensor,Sens. Typ,Location,Cont. Typ,Farbe,Address,loc,point,lat,lon,altitude
0,70B3D500700016DA,Senseno Ultrasonic,An den Roethen 53,Glas (3 Kammern),white,"An den Roethen 53, Frankfurt","(An den Röthen 53, 60389 Frankfurt am Main, Germany, (50.13464, 8.710139999999999))","(50.13464, 8.710139999999999, 0.0)",50.134640,8.710140,0.0
1,70B3D500700016DE,Senseno Ultrasonic,Hungener Str. 6,Glas (3 Kammern),brown,"Hungener Str. 6, Frankfurt","(Hungener Str. 6, 60389 Frankfurt am Main, Germany, (50.13320849999999, 8.6974312))","(50.13320849999999, 8.6974312, 0.0)",50.133208,8.697431,0.0
2,70B3D500700016DF,Senseno Ultrasonic,Standort unbekannt,Glas (3 Kammern),white,"Standort unbekannt, Frankfurt","(Frankfurt, Germany, (50.1109221, 8.6821267))","(50.1109221, 8.6821267, 0.0)",50.110922,8.682127,0.0
3,70B3D500700016E0,Senseno Ultrasonic,Gronauer Str. 39,Glas (3 Kammern),green,"Gronauer Str. 39, Frankfurt","(Gronauer Str. 39, 60385 Frankfurt am Main, Germany, (50.1288563, 8.7109253))","(50.1288563, 8.7109253, 0.0)",50.128856,8.710925,0.0
4,70B3D500700016E5,Senseno Ultrasonic,Wetteraustr. 1,Glas (3 Kammern),green,"Wetteraustr. 1, Frankfurt","(Wetteraustraße 1, 60389 Frankfurt am Main, Germany, (50.12938579999999, 8.7007069))","(50.12938579999999, 8.7007069, 0.0)",50.129386,8.700707,0.0
...,...,...,...,...,...,...,...,...,...,...,...
72,70B3D50070001782,Senseno Ultrasonic,Gronauer Str. 39,Glas (3 Kammern),brown,"Gronauer Str. 39, Frankfurt","(Gronauer Str. 39, 60385 Frankfurt am Main, Germany, (50.1288563, 8.7109253))","(50.1288563, 8.7109253, 0.0)",50.128856,8.710925,0.0
73,70B3D50070001786,Senseno Ultrasonic,Wetteraustr. 1,Glas (3 Kammern),brown,"Wetteraustr. 1, Frankfurt","(Wetteraustraße 1, 60389 Frankfurt am Main, Germany, (50.12938579999999, 8.7007069))","(50.12938579999999, 8.7007069, 0.0)",50.129386,8.700707,0.0
74,70B3D50070001787,Senseno Ultrasonic,Seckbacher Landstr.74,Glas (3 Kammern),green,"Seckbacher Landstr.74, Frankfurt","(Seckbacher Landstraße 74, 60389 Frankfurt am Main, Germany, (50.13584609999999, 8.7126858))","(50.13584609999999, 8.7126858, 0.0)",50.135846,8.712686,0.0
75,70B3D50070001788,Senseno Ultrasonic,Walter-Hesselbach-Straße 210,Glas (3 Kammern),green,"Walter-Hesselbach-Straße 210, Frankfurt","(Walter-Hesselbach-Straße 210, 60389 Frankfurt am Main, Germany, (50.14207099999999, 8.6980182))","(50.14207099999999, 8.6980182, 0.0)",50.142071,8.698018,0.0


# Plot it on a map

In [56]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap

In [64]:
output_file("../data/modeling/plots/bokeh.html")

map_options = GMapOptions(lat = 50.13, lng = 8.73, map_type = "roadmap", zoom = 12)
bokeh_plot = gmap(AUTH_KEY, map_options, title = "Container locations")

source = ColumnDataSource(
    data = dict(lat = geo_df['lat'],
              lon = geo_df['lon'])
)
bokeh_plot.circle(x = "lon", y = "lat", size = 10, fill_alpha = 0.8, source = source)
show(bokeh_plot)

Was hier direkt auffällt: 
   - Bornheim = Wohngebiet
   - Fechenheim = Industriegebiet
    
--> Feature hinzufügen

Der Punkt am HBF ist wahrscheinlich der Container mit "Standort unbekannt"

# Join Positions with Sensor Data

In [24]:
#8.76

geo_df["ist_industriegebiet"] = geo_df["lon"]>8.74
geo_df["ist_wohngebiet"] = geo_df["lon"]<8.74

In [25]:
geo_df

,Sensor,Sens. Typ,Location,Cont. Typ,Farbe,Address,loc,point,lat,lon,altitude,ist_industriegebiet,ist_wohngebiet
0,70B3D500700016DA,Senseno Ultrasonic,An den Roethen 53,Glas (3 Kammern),white,"An den Roethen 53, Frankfurt","(An den Röthen 53, 60389 Frankfurt am Main, Germany, (50.13464, 8.710139999999999))","(50.13464, 8.710139999999999, 0.0)",50.134640,8.710140,0.0,False,True
1,70B3D500700016DE,Senseno Ultrasonic,Hungener Str. 6,Glas (3 Kammern),brown,"Hungener Str. 6, Frankfurt","(Hungener Str. 6, 60389 Frankfurt am Main, Germany, (50.13320849999999, 8.6974312))","(50.13320849999999, 8.6974312, 0.0)",50.133208,8.697431,0.0,False,True
2,70B3D500700016DF,Senseno Ultrasonic,Standort unbekannt,Glas (3 Kammern),white,"Standort unbekannt, Frankfurt","(Frankfurt, Germany, (50.1109221, 8.6821267))","(50.1109221, 8.6821267, 0.0)",50.110922,8.682127,0.0,False,True
3,70B3D500700016E0,Senseno Ultrasonic,Gronauer Str. 39,Glas (3 Kammern),green,"Gronauer Str. 39, Frankfurt","(Gronauer Str. 39, 60385 Frankfurt am Main, Germany, (50.1288563, 8.7109253))","(50.1288563, 8.7109253, 0.0)",50.128856,8.710925,0.0,False,True
4,70B3D500700016E5,Senseno Ultrasonic,Wetteraustr. 1,Glas (3 Kammern),green,"Wetteraustr. 1, Frankfurt","(Wetteraustraße 1, 60389 Frankfurt am Main, Germany, (50.12938579999999, 8.7007069))","(50.12938579999999, 8.7007069, 0.0)",50.129386,8.700707,0.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,70B3D50070001782,Senseno Ultrasonic,Gronauer Str. 39,Glas (3 Kammern),brown,"Gronauer Str. 39, Frankfurt","(Gronauer Str. 39, 60385 Frankfurt am Main, Germany, (50.1288563, 8.7109253))","(50.1288563, 8.7109253, 0.0)",50.128856,8.710925,0.0,False,True
73,70B3D50070001786,Senseno Ultrasonic,Wetteraustr. 1,Glas (3 Kammern),brown,"Wetteraustr. 1, Frankfurt","(Wetteraustraße 1, 60389 Frankfurt am Main, Germany, (50.12938579999999, 8.7007069))","(50.12938579999999, 8.7007069, 0.0)",50.129386,8.700707,0.0,False,True
74,70B3D50070001787,Senseno Ultrasonic,Seckbacher Landstr.74,Glas (3 Kammern),green,"Seckbacher Landstr.74, Frankfurt","(Seckbacher Landstraße 74, 60389 Frankfurt am Main, Germany, (50.13584609999999, 8.7126858))","(50.13584609999999, 8.7126858, 0.0)",50.135846,8.712686,0.0,False,True
75,70B3D50070001788,Senseno Ultrasonic,Walter-Hesselbach-Straße 210,Glas (3 Kammern),green,"Walter-Hesselbach-Straße 210, Frankfurt","(Walter-Hesselbach-Straße 210, 60389 Frankfurt am Main, Germany, (50.14207099999999, 8.6980182))","(50.14207099999999, 8.6980182, 0.0)",50.142071,8.698018,0.0,False,True


In [26]:
geo_df.to_csv('../data/preprocessed/geo_data.csv', index=False)